In [1]:
import os
from torch import optim, nn, utils, Tensor
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
import lightning.pytorch as pl

# define any number of nn.Modules (or use your current ones)
encoder = nn.Sequential(nn.Linear(28 * 28, 64), nn.ReLU(), nn.Linear(64, 3))
decoder = nn.Sequential(nn.Linear(3, 64), nn.ReLU(), nn.Linear(64, 28 * 28))


# define the LightningModule
class LitAutoEncoder(pl.LightningModule):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        # it is independent of forward
        x, y = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = nn.functional.mse_loss(x_hat, x)
        # Logging to TensorBoard (if installed) by default
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        return optimizer


# init the autoencoder
autoencoder = LitAutoEncoder(encoder, decoder)

In [2]:
dataset = MNIST(os.getcwd(), download=True, transform=ToTensor())
train_loader = utils.data.DataLoader(dataset)

100%|██████████| 9912422/9912422 [00:00<00:00, 12186736.97it/s]


Extracting C:\Users\HansW\mitwworkspace\MakeAIWork2\practica\PyTorch\MNIST\raw\train-images-idx3-ubyte.gz to C:\Users\HansW\mitwworkspace\MakeAIWork2\practica\PyTorch\MNIST\raw



100%|██████████| 28881/28881 [00:00<?, ?it/s]

Extracting C:\Users\HansW\mitwworkspace\MakeAIWork2\practica\PyTorch\MNIST\raw\train-labels-idx1-ubyte.gz to C:\Users\HansW\mitwworkspace\MakeAIWork2\practica\PyTorch\MNIST\raw




100%|██████████| 1648877/1648877 [00:00<00:00, 16473899.79it/s]


Extracting C:\Users\HansW\mitwworkspace\MakeAIWork2\practica\PyTorch\MNIST\raw\t10k-images-idx3-ubyte.gz to C:\Users\HansW\mitwworkspace\MakeAIWork2\practica\PyTorch\MNIST\raw



100%|██████████| 4542/4542 [00:00<?, ?it/s]


Extracting C:\Users\HansW\mitwworkspace\MakeAIWork2\practica\PyTorch\MNIST\raw\t10k-labels-idx1-ubyte.gz to C:\Users\HansW\mitwworkspace\MakeAIWork2\practica\PyTorch\MNIST\raw



In [3]:
trainer = pl.Trainer(limit_train_batches=100, max_epochs=1)
trainer.fit(model=autoencoder, train_dataloaders=train_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
C:\Users\HansW\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\lightning\pytorch\trainer\connectors\logger_connector\logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(
Missing logger folder: C:\Users\HansW\mitwworkspace\MakeAIWork2\practica\PyTorch\lightning_logs

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


In [5]:
checkpoint = "./lightning_logs/version_0/checkpoints/epoch=0-step=100.ckpt"
autoencoder = LitAutoEncoder.load_from_checkpoint(checkpoint, encoder=encoder, decoder=decoder)

In [6]:
encoder = autoencoder.encoder
encoder.eval()

Sequential(
  (0): Linear(in_features=784, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=3, bias=True)
)

In [7]:
fake_image_batch = Tensor(4, 28 * 28)
embeddings = encoder(fake_image_batch)
print("⚡" * 20, "\nPredictions (4 image embeddings):\n", embeddings, "\n", "⚡" * 20)

⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡ 
Predictions (4 image embeddings):
 tensor([[-0.0164,  0.0255,  0.0290],
        [-0.0164,  0.0255,  0.0290],
        [-0.0164,  0.0255,  0.0290],
        [-0.0164,  0.0255,  0.0290]], grad_fn=<AddmmBackward0>) 
 ⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡
